In [ ]:
!pip install scipy


In [ ]:
import ta.momentum
import yfinance as yf
import pandas as pd
import ta as ta
from scipy.signal import argrelextrema
import numpy as np

# Download data with timezone information
spy = yf.download("CMS", period="1y", interval="1d")
rsi_input = spy["Close"].squeeze()

spy["RSI"] = ta.momentum.RSIIndicator(rsi_input, window=14).rsi()

def get_swing_lows(prices, order=5):
    idx = argrelextrema(prices.values, np.less_equal, order=order)[0]
    return prices.iloc[idx]

swing_lows = get_swing_lows(spy['Close'], order=5)

# Identify bullish divergence
bullish_divergences = []

for i in range(1, len(swing_lows)):
    prev_date = swing_lows.index[i - 1]
    curr_date = swing_lows.index[i]

    # Extract scalar values using .values[0]
    prev_price = spy.loc[prev_date, 'Close'].values[0]
    curr_price = spy.loc[curr_date, 'Close'].values[0]

    prev_rsi = spy.loc[prev_date, 'RSI'].values[0]
    curr_rsi = spy.loc[curr_date, 'RSI'].values[0]

    if curr_price < prev_price and curr_rsi > prev_rsi and (curr_rsi < 40 and prev_rsi < 35):
        # Store the full datetime index instead of just the date
        bullish_divergences.append((prev_date, curr_date))
print('bullish divergences ', bullish_divergences)

# Analyze price movements following divergences
successful_trades = 0
total_trades = len(bullish_divergences)

for divergence in bullish_divergences:
    # Use the datetime index directly
    start_date = divergence[1]
    
    # Get the index of the start date
    start_idx = spy.index.get_loc(start_date)
    
    # Get the next 10 trading periods (if available)
    if start_idx + 10 < len(spy):
        # Get the base price (price at divergence point)
        base_price = spy.iloc[start_idx]['Close']
        
        # Get the next 10 periods of prices
        next_10_periods = spy.iloc[start_idx+1:start_idx+11]['Close']
        
        # Check if price increased by 2% at any point
        max_price = next_10_periods.max()
        price_increase = float((max_price - base_price) / base_price * 100)
        
        if price_increase >= 2:
            successful_trades += 1

# Calculate and print results
success_rate = (successful_trades / total_trades * 100) if total_trades > 0 else 0
print(f'\nResults:')
print(f'Total divergences found: {total_trades}')
print(f'Successful trades (2%+ increase): {successful_trades}')
print(f'Success rate: {success_rate:.2f}%')